In [ ]:
# Step 1: Install Tesseract and required libraries
!apt-get install -y tesseract-ocr
!pip install pytesseract opencv-python gradio
!pip install --upgrade gradio
!apt-get install -y tesseract-ocr-dan
!pip install rapidfuzz

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,644 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Define paths
base_dir = '/content/drive/MyDrive/Piktogrammer'  # Update this path
img_height, img_width = 128, 128  # Size to which images will be resized
batch_size = 32

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Rescale pixel values
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Build model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Save model
model.save('pictogram_model.h5')


In [ ]:
import os
import cv2
import pytesseract
import gradio as gr
import numpy as np
from google.colab import drive
from pathlib import Path
from PIL import Image

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Set the path to the pictogram folder
pictogram_folder = '/content/drive/MyDrive/Piktogrammer'  # Update this path

# Specify Tesseract command path if needed
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'  # Adjust path if different in Colab

# Step 3: Function to extract text from uploaded schedule image
def extract_text_from_image(image):
    image = np.array(image)  # Convert PIL image to NumPy array
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for Tesseract
    gray_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)  # Convert to grayscale for better OCR performance
    _, thresh_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY_INV)  # Apply thresholding

    # Step 3.1: Contour detection
    contours, _ = cv2.findContours(thresh_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = []

    # Step 3.2: Filtering small contours
    MIN_CONTOUR_AREA = 100  # Adjust this threshold as necessary
    for contour in contours:
        if cv2.contourArea(contour) < MIN_CONTOUR_AREA:
            continue  # Skip small contours
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append((x, y, w, h))

    # Step 3.3: Sort bounding boxes
    bounding_boxes.sort(key=lambda box: (box[1], box[0]))  # Sort by y, then x

    # Step 3.4: Extract text from each bounding box
    detected_texts = set()  # Using a set to avoid duplicates
    for box in bounding_boxes:
        x, y, w, h = box
        roi = image[y:y+h, x:x+w]  # Region of interest
        text = pytesseract.image_to_string(roi, lang='dan').strip()  # Specify Danish language
        if text:  # Only add non-empty results
            detected_texts.add(text)

    # Convert set back to list and maintain order
    detected_texts = sorted(list(detected_texts), key=lambda t: bounding_boxes[bounding_boxes.index(box)][1])
    print(f"Detected Texts: {detected_texts}")  # Debugging print
    return detected_texts

# Step 4: Function to match extracted text with pictograms
def match_pictograms(detected_text):
    matched_images = []
    matched_filenames = []  # Store filenames for output
    for text in detected_text:
        # Format text to match folder naming conventions
        category_folder = Path(pictogram_folder) / text.lower().replace(" ", "_")  # Format text
        print(f"Attempting to locate folder: {category_folder}")  # Debugging print
        if category_folder.exists() and category_folder.is_dir():
            print(f"Found matching folder: {category_folder}")  # Debugging print
            for img_file in os.listdir(category_folder):
                img_path = category_folder / img_file
                matched_images.append(img_path)
                matched_filenames.append(img_file)  # Append file name
        else:
            matched_images.append("No matching category found.")
            matched_filenames.append("No matching category found.")
            print(f"No matching category found for: '{text}'")  # Debugging print for specific text
    return matched_images, matched_filenames

# Step 5: Function to process uploaded image and display results
def process_image(uploaded_image):
    detected_text = extract_text_from_image(uploaded_image)  # Use OCR
    print(f"Detected Text for Matching: {detected_text}")  # Print detected text for debugging
    matched_pictograms, matched_filenames = match_pictograms(detected_text)
    matched_images = []
    for img in matched_pictograms:
        if isinstance(img, str) and img.startswith("No matching"):
            matched_images.append(img)
        else:
            # Verify the path and check if the image loads correctly
            print(f"Loading image from: {img}")  # Debugging path print
            img_data = cv2.imread(str(img))
            if img_data is not None:
                # Convert image from OpenCV to PIL format for Gradio compatibility
                pil_image = Image.fromarray(cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB))
                matched_images.append(pil_image)  # Append image without filename
            else:
                matched_images.append("Image could not be read.")
                print(f"Error reading image: {img}")  # Debugging print
    return detected_text, matched_images, matched_filenames

# Step 6: Gradio Interface
interface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil", label="Upload Schedule Image"),
    outputs=[
        gr.Textbox(label="Detected Texts"),
        gr.Gallery(label="Matched Pictograms"),
        gr.Textbox(label="Matched File Names")
    ],
    title="Weekplanner_V5 with OCR",
    description="Upload an image of the child's schedule. The AI will extract text and display matching pictograms."
)

# Step 7: Launch the Gradio app
interface.launch(share=True)


Mounted at /content/drive
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2ed3911a7b8208c276.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
